<a href="https://colab.research.google.com/github/Joseph669/TestRepo/blob/master/svmtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# 挂载Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("/content/drive/HRV")
!ls

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!pwd
import os
os.chdir("/content/drive/HRV")
!ls

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
iris = load_iris()
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=0)
print("Size of training set:{} size of testing set:{}".format(X_train.shape[0],X_test.shape[0]))

X_trainval,X_test,y_trainval,y_test = train_test_split(iris.data,iris.target,random_state=0)
X_train,X_val,y_train,y_val = train_test_split(X_trainval,y_trainval,random_state=1)
print("Size of training set:{} size of validation set:{} size of teseting set:{}".format(X_train.shape[0],X_val.shape[0],X_test.shape[0]))

best_score = 0.0
for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        svm = SVC(gamma=gamma,C=C)
        scores = cross_val_score(svm,X_trainval,y_trainval,cv=5) #5折交叉验证
        score = scores.mean() #取平均数
        if score > best_score:
            best_score = score
            best_parameters = {"gamma":gamma,"C":C}
svm = SVC(**best_parameters)
svm.fit(X_trainval,y_trainval)
test_score = svm.score(X_test,y_test)
print("Best score on validation set:{:.2f}".format(best_score))
print("Best parameters:{}".format(best_parameters))
print("Score on testing set:{:.2f}".format(test_score))

Size of training set:112 size of testing set:38
Size of training set:84 size of validation set:28 size of teseting set:38
Best score on validation set:0.97
Best parameters:{'gamma': 0.01, 'C': 100}
Score on testing set:0.97


In [4]:
from sklearn.model_selection import GridSearchCV

#把要调整的参数以及其候选值 列出来；
param_grid = {"gamma":[0.001,0.01,0.1,1,10,100],
             "C":[0.001,0.01,0.1,1,10,100]}
print("Parameters:{}".format(param_grid))

grid_search = GridSearchCV(SVC(),param_grid,cv=5) #实例化一个GridSearchCV类
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=10)
grid_search.fit(X_train,y_train) #训练，找到最优的参数，同时使用最优的参数实例化一个新的SVC estimator。
print("Test set score:{:.2f}".format(grid_search.score(X_test,y_test)))
print("Best parameters:{}".format(grid_search.best_params_))
print("Best score on train set:{:.2f}".format(grid_search.best_score_))

Parameters:{'gamma': [0.001, 0.01, 0.1, 1, 10, 100], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
Test set score:0.97
Best parameters:{'C': 10, 'gamma': 0.1}
Best score on train set:0.98


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [0]:
#from get_train_feats import get_train_feats
from get_test_feats import get_test_feats
from new_get_train_feats import new_get_train_feats

new_get_train_feats() 
#get_train_feats()  # video1,3  NO4-NO18
get_test_feats()   # video2    NO4-NO18


In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

file_name = 'new_train_feats_data.csv'
#file_name = 'test_feats_data.csv'
# feats_tuple     = sample(range(0,10), 3)
feats_tuple     =  [4,6]

file_data = np.array(pd.read_csv(file_name))
feats     = file_data[:, feats_tuple]
#feats     = preprocessing.StandardScaler().fit_transform(feats)
label     = file_data[:, -1]
x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.1, random_state=110)
clf = SVC(C=0.5, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参


print('----------------------------------------------------------')        
print('----------------------------------------------------------')
print('train accuracy:' + str(clf.score(x_train, y_train)))
print(confusion_matrix(y_train, clf.predict(x_train)))
print('test accuracy:' + str(clf.score(x_test, y_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))


----------------------------------------------------------
----------------------------------------------------------
train accuracy:0.6
[[46  4]
 [36 14]]
test accuracy:0.6666666666666666
[[6 0]
 [4 2]]


In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (10,20)


file_name = 'new_train_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
x_train     = file_data[:, feats_tuple]
#x_train     = preprocessing.StandardScaler().fit_transform(x_train)
y_train     = file_data[:, -1]
#x_train,x_test,y_train,y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=2)

file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
x_test     = file_data[:, feats_tuple]
#x_test     = preprocessing.StandardScaler().fit_transform(x_test)
y_test     = file_data[:, -1]

clf = SVC(C=100, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参


print('----------------------------------------------------------')        
print('----------------------------------------------------------')
print('train accuracy:' + str(clf.score(x_train, y_train)))
print(confusion_matrix(y_train, clf.predict(x_train)))
print('test accuracy:' + str(clf.score(x_test, y_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))


----------------------------------------------------------
----------------------------------------------------------
train accuracy:0.5446428571428571
[[35 21]
 [30 26]]
test accuracy:0.37735849056603776
[[ 9 18]
 [15 11]]


In [0]:
from sklearn.neighbors import KNeighborsClassifier 
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample

#feats_tuple     = sample(range(0,10), 3)
#feats_tuple     =(0,1,4,10,16)
#feats_tuple = [1,  2,  3,  7, 16]
feats_tuple = [0, 3, 4, 5, 6]
file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
print(file_data.shape)
x_train     = file_data[:, feats_tuple]
#x_train     = preprocessing.StandardScaler().fit_transform(x_train)
y_train     = file_data[:, -1]

file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
x_test     = file_data[:, feats_tuple]
#x_test     = preprocessing.StandardScaler().fit_transform(x_test)
y_test     = file_data[:, -1]
x_train,x_test,y_train,y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=2)
#knn = KNeighborsClassifier().fit(x_train, y_train)
knn = SVC(C=1, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参

y_predict = knn.predict(x_test) 
#probility=knn.predict_proba(x_test)  
#neighborpoint=knn.kneighbors(x_test[-1].reshape(1, -1),5,False)
score=knn.score(x_test,y_test,sample_weight=None)
print('train Accuracy:'+str(knn.score(x_train,y_train,sample_weight=None)))
print(confusion_matrix(y_train, knn.predict(x_train)))
print('test Accuracy:'+str(score))
print(confusion_matrix(y_test, knn.predict(x_test)))

(53, 22)
train Accuracy:1.0
[[22  0]
 [ 0 25]]
test Accuracy:0.16666666666666666
[[0 5]
 [0 1]]


In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample
from itertools import combinations


file_name = 'train_feats_data.csv'
file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,4, 7,12)

def change_feats(feats_tuple):    
    feats     = file_data[:, feats_tuple]
    #feats     = preprocessing.StandardScaler().fit_transform(feats)
    label     = file_data[:, -1]
    x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.3, random_state=2)
    clf = SVC(C=1000, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参
    # acc_dict[str(feats)] =  str(clf.score(x_train, y_train)) + ', ' + str(clf.score(x_test, y_test))
    print('-------------------'+str(feats_tuple)+'--------------------------')
    print('train accuracy:' + str(clf.score(x_train, y_train)))
    print(confusion_matrix(y_train, clf.predict(x_train)))
    print('test accuracy:' + str(clf.score(x_test, y_test)))
    print(confusion_matrix(y_test, clf.predict(x_test)))

    
for feats in  itertools.combinations(feats_tuple, 4):
    change_feats(feats)
    
    

In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample
from itertools import combinations


file_name = 'new_train_feats_data.csv'
#file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))


def change_data(feats_tuple, input_row=None): 
    if input_row:
        global file_data
        data_tuple = np.array(input_row)
        data_tuple = np.concatenate((data_tuple, data_tuple+15, data_tuple+30, data_tuple+45,
                                    data_tuple+60, data_tuple+75, data_tuple+90, data_tuple+105))
        select_file_data = file_data[data_tuple, :]
        feats     = select_file_data[:, feats_tuple]
        label     = select_file_data[:, -1]
        print('-------------------'+str(input_row)+'--------------------------')
    else:
        feats     = file_data[:, feats_tuple]
        label     = file_data[:, -1]
    #feats     = preprocessing.StandardScaler().fit_transform(feats)
    #x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.3, random_state=2)
    #clf = SVC(C=1000, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参
    # acc_dict[str(feats)] =  str(clf.score(x_train, y_train)) + ', ' + str(clf.score(x_test, y_test))
    #file_name = 'new_train_feats_data.csv'
    #file_data = np.array(pd.read_csv(file_name))
    x_train     = file_data[:, feats_tuple]
    x_train     = preprocessing.StandardScaler().fit_transform(x_train)
    y_train     = file_data[:, -1]

    file_name = 'test_feats_data.csv'
    file_data = np.array(pd.read_csv(file_name))
    x_test     = file_data[:, feats_tuple]
    x_test     = preprocessing.StandardScaler().fit_transform(x_test)
    y_test     = file_data[:, -1]
    clf = SVC(C=1000, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参
    #print('-------------------'+str(feats_tuple)+'--------------------------')
    print('train accuracy:' + str(clf.score(x_train, y_train)))
    print(confusion_matrix(y_train, clf.predict(x_train)))
    print('test accuracy:' + str(clf.score(x_test, y_test)))
    print(confusion_matrix(y_test, clf.predict(x_test)))

# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,4, 7,12)    
#for feats in  itertools.combinations(feats_tuple, 4):
#    change_feats(feats)
data_tuple = range(0,15)    
for data_comb in  itertools.combinations(data_tuple, 12):
    change_data(feats_tuple,data_comb)

    

In [0]:
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt 
import pandas as pd
from random import sample
from itertools import combinations


file_name = 'train_feats_data.csv'
file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))


def change_data(feats_tuple, input_row=None): 
    if input_row:
        data_tuple = np.array(input_row)
        data_tuple = np.concatenate((data_tuple, data_tuple+15, data_tuple+30, data_tuple+45))
        select_file_data = file_data[data_tuple, :]
        feats     = select_file_data[:, feats_tuple]
        label     = select_file_data[:, -1]
        print('-------------------'+str(input_row)+'--------------------------')
    else:
        feats     = file_data[:, feats_tuple]
        label     = file_data[:, -1]
    #feats     = preprocessing.StandardScaler().fit_transform(feats)
    x_train,x_test,y_train,y_test = train_test_split(feats, label, test_size=0.3, random_state=2)
    clf = SVC(C=1000, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train) #调参
    # acc_dict[str(feats)] =  str(clf.score(x_train, y_train)) + ', ' + str(clf.score(x_test, y_test))
    #print('-------------------'+str(feats_tuple)+'--------------------------')
    print('train accuracy:' + str(clf.score(x_train, y_train)))
    print(confusion_matrix(y_train, clf.predict(x_train)))
    print('test accuracy:' + str(clf.score(x_test, y_test)))
    print(confusion_matrix(y_test, clf.predict(x_test)))

# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,4, 7,12)    
#for feats in  itertools.combinations(feats_tuple, 4):
#    change_feats(feats)
data_tuple = range(0,15)    
for data_comb in  itertools.combinations(data_tuple, 12):
    change_data(feats_tuple,data_comb)

    

In [0]:
# feats_tuple     = sample(range(0,10), 3)
feats_tuple     = (0,1,4, 7,12)import numpy as np
import pandas as pd
import csv,os
from sklearn import preprocessing


output_file = 'subtraction.csv'
if os.path.exists(output_file):
    os.remove(output_file)
    
relax = (np.array(pd.read_csv('relax.csv')))[:, :]
stress = (np.array(pd.read_csv('stress.csv')))[:, :]

subtraction = stress - relax
#subtraction = preprocessing.MinMaxScaler().fit_transform(subtraction)
for row in subtraction:
    out = open(output_file, 'a', newline='')
    csv_write = csv.writer(out,dialect='excel')
    csv_write.writerow(row)

In [0]:
from sklearn.feature_selection import SelectKBest, chi2,f_regression
from sklearn.neighbors import KNeighborsClassifier 
import numpy as np
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
from random import sample

file_name = 'new_train_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
x_train     = file_data[:, :-1]
y_train     = file_data[:, -1].reshape((-1,1))

#scaler =  preprocessing.MinMaxScaler()
#scaler =  preprocessing.StandardScaler()

#x_train = scaler.fit_transform(x_train)
feats_tuple = SelectKBest(f_regression, k=15).fit(x_train, y_train).get_support(indices = True)
print(feats_tuple)
feats_tuple = [0, 3, 4, 5, 6]
d
x_train = x_train[:, feats_tuple]
#x_train = scaler.fit_transform(x_train)

#knn = KNeighborsClassifier().fit(x_train, y_train)
knn = SVC(C=10, coef0=0, kernel='rbf', gamma='auto').fit(x_train, y_train)
file_name = 'test_feats_data.csv'
file_data = np.array(pd.read_csv(file_name))
#x_test     = (scaler.transform(file_data[:,:-1]))[:, feats_tuple]
x_test     = file_data[:,:-1][:, feats_tuple]
y_test     = file_data[:, -1].reshape((-1,1))
#x_train,x_test,y_train,y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=5)
                       
y_predict = knn.predict(x_test) 
score=knn.score(x_test,y_test,sample_weight=None)
print('train Accuracy:'+str(knn.score(x_train,y_train,sample_weight=None)))
print(confusion_matrix(y_train, knn.predict(x_train)))
print('test Accuracy:'+str(score))
print(confusion_matrix(y_test, knn.predict(x_test)))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 16 17]
train Accuracy:0.9732142857142857
[[56  0]
 [ 3 53]]
test Accuracy:0.4528301886792453
[[ 7 20]
 [ 9 17]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
 
%matplotlib inline 
 
data = pd.read_csv("census.csv")
 
# 将数据切分成特征和标签
income_raw = data['income']
features_raw = data.drop('income', axis=1)
 
# 显示部分数据
# display(features_raw.head(n=1))
 
# 因为原始数据中的，capital-gain 和 capital-loss的倾斜度非常高，所以要是用对数转换。
skewed = ['capital-gain', 'capital-loss']
features_raw[skewed] = data[skewed].apply(lambda x: np.log(x + 1))
 
# 归一化数字特征,是为了保证所有的特征均被平等的对待
scaler = MinMaxScaler()
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
features_raw[numerical] = scaler.fit_transform(data[numerical])
# display(features_raw.head(n=1))
 
# 独热编码，将非数字的形式转化为数字
features = pd.get_dummies(features_raw)
income = income_raw.replace(['>50K', ['<=50K']], [1, 0])
 
# 切分数据集
X_train, X_test, y_train, y_test = train_test_split(features, income, test_size=0.2, random_state=0)
 
# Adaboost
from sklearn.ensemble import AdaBoostClassifier
clf_Ada = AdaBoostClassifier(random_state=0)
 
# 决策树
from sklearn.tree import DecisionTreeClassifier
clf_Tree = DecisionTreeClassifier(random_state=0)
 
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier()
 
# SVM
from sklearn.svm import SVC
clf_svm = SVC(random_state=0)
 
# Logistic
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state=0)
 
# 随机森林
from sklearn.ensemble import RandomForestClassifier
clf_forest = RandomForestClassifier(random_state=0)
 
# GBDT
from sklearn.ensemble import GradientBoostingClassifier
clf_gbdt = GradientBoostingClassifier(random_state=0)
 
# GaussianNB
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()
 
scorer = make_scorer(accuracy_score)
 
# 参数调优
 
kfold = KFold(n_splits=10)
# 决策树
parameter_tree = {'max_depth': xrange(1, 10)}
grid = GridSearchCV(clf_Tree, parameter_tree, scorer, cv=kfold)
grid = grid.fit(X_train, y_train)
 
print "best score: {}".format(grid.best_score_)
display(pd.DataFrame(grid.cv_results_).T)


#段落

In [0]:
from panTompkins import panTompkins
from timeDomain import timeDomain
from frequencyDomain import frequencyDomain
import gc, os, platform, csv
import pandas as pd
import numpy as np
from poincare import eclipseFittingMethod, hraMethod, correlationCoef

output_file = 'new_train_feats_data.csv'

def new_get_train_feats():
    if os.path.exists(output_file):
        os.remove(output_file)

    get_all_tester_feats('relax')     
    get_all_tester_feats('stress')        


def get_all_tester_feats(file_type):
    assert file_type =='relax' or file_type == 'stress'

    if file_type == 'relax':
        input_file = 'video1.csv'
    elif file_type == 'stress':
        input_file = 'video3.csv'

    for num in range(4, 19):
        if num != 10:
            print('-----------------------'+str(num)+'---------------------------')
            abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + input_file
            if (platform.system() == 'Windows'):
                abs_path.replace('/', '\\')
            all_data = (np.array(pd.read_csv(abs_path)))[:,1]
            get_single_tester_feats(all_data[     0: 45000], file_type)
            get_single_tester_feats(all_data[ 45000: 90000], file_type)
            get_single_tester_feats(all_data[ 90000:135000], file_type)
            get_single_tester_feats(all_data[135000:      ], file_type)
            del all_data
            gc.collect()

def get_single_tester_feats(csv_data, file_type):
    label = 1 if file_type=='stress' else 0
    R_peak_locs = panTompkins(ECG=csv_data, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300
    timeDomainFeats = timeDomain(RR_interval_series)
    freqDomainFeats = frequencyDomain(RR_interval_series)
    fittingFeats = eclipseFittingMethod(RR_interval_series)
    hraFeats = hraMethod(RR_interval_series)
    feats_list = [val for val in timeDomainFeats.values()] + \
        [val for val in freqDomainFeats.values()] + \
        [val for val in fittingFeats.values()] + \
        [val for val in hraFeats.values()] + \
        [correlationCoef(RR_interval_series), label]
    
    if os.path.exists(output_file):
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_list)
    else:    # 不存在文件则写入header+data
        t_keys = [key for key in timeDomainFeats.keys()]
        f_keys = [key for key in freqDomainFeats.keys()]
        feats_header_list = [key for key in timeDomainFeats.keys()] + \
            [key for key in freqDomainFeats.keys()] + \
            [key for key in fittingFeats.keys()] + \
            [key for key in hraFeats.keys()] + \
            ['CorrCoef','label']
        out = open(output_file, 'a', newline='')
        csv_write = csv.writer(out,dialect='excel')
        csv_write.writerow(feats_header_list)
        csv_write.writerow(feats_list)
        
new_get_train_feats()        

# fig函数

In [0]:
# 画图所需数据库的数据
from panTompkins import panTompkins
import gc, os, platform, csv
import pandas as pd
import numpy as np

input_file  = ('video1.csv', 'video3.csv')
output_file = 'train_fig_data.csv'
cnt = 1

def get_fig_data(csv_data):
    global cnt, output_file
    R_peak_locs = panTompkins(ECG=csv_data, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300

    fig_list = [cnt, str(R_peak_locs), str(RR_interval_series.tolist())]
    out = open(output_file, 'a', newline='')
    csv_write = csv.writer(out,dialect='excel')
    if not os.path.exists(output_file):     # write header
        fig_header_list = ['index', 'ECG', 'R_location', 'RR_interval']
        csv_write.writerow(fig_header_list)  
    csv_write.writerow(fig_list)            # write feats
    out.close()
    cnt += 1

    
if __name__ == '__main__':
    if os.path.exists(output_file):
        os.remove(output_file)
        
    for videocsv in input_file:
        for num in range(4, 19):
            print('-----------------------'+str(num)+'---------------------------')
            abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + videocsv
            if (platform.system() == 'Windows'):
                abs_path.replace('/', '\\')
            all_data = (np.array(pd.read_csv(abs_path)))[:,1]
            get_fig_data(all_data[     0: 45000])
            get_fig_data(all_data[ 45000: 90000])
            get_fig_data(all_data[ 90000:135000])
            get_fig_data(all_data[135000:      ])
            del all_data
            gc.collect()

In [0]:
# 画图所需数据库的数据
from panTompkins import panTompkins
import gc, os, platform, csv
import pandas as pd
import numpy as np

input_file  = 'video2.csv'
output_file = 'test_fig_data.csv'
cnt = 1

def get_fig_data(csv_data):
    global cnt, output_file
    R_peak_locs = panTompkins(ECG=csv_data, fs=300, plot=0)
    RR_interval_series = (np.diff(R_peak_locs)) / 300

    fig_list = [cnt, str(R_peak_locs), str(RR_interval_series.tolist())]
    out = open(output_file, 'a', newline='')
    csv_write = csv.writer(out,dialect='excel')
    if not os.path.exists(output_file):     # write header
        fig_header_list = ['index', 'R_location', 'RR_interval']
        csv_write.writerow(fig_header_list)  
    csv_write.writerow(fig_list)            # write feats
    out.close()
    cnt += 1


if __name__ == '__main__':
    if os.path.exists(output_file):
        os.remove(output_file)

    for num in range(4, 19):
        print('-----------------------'+str(num)+'---------------------------')
        abs_path = os.getcwd() + '/Data/NO'  + str(num) + '/' + input_file
        if (platform.system() == 'Windows'):
            abs_path.replace('/', '\\')
        all_data = (np.array(pd.read_csv(abs_path)))[:,1]
        get_fig_data(all_data[     0: 54000])
        get_fig_data(all_data[ 54000: 90000])
        get_fig_data(all_data[ 90000:126000])
        get_fig_data(all_data[126000:      ])
        del all_data
        gc.collect()



---

